In [1]:
%reload_ext autoreload
%autoreload 2

## Script

### Initialize, train and test function

In [18]:
from npfd.models.base import HiddenMarkovModel

def train_evaluate(plot_results=False, 
                   log=True,
                   **kwargs):
    
    X_train, X_test, y_train, y_test = data.dataset.make_dataset('dmps', 
                                                                 clean_interim_dir=True, 
                                                                 test_size=0.3)
    

    model = HiddenMarkovModel()
    
    model.initialize(X_train, init_method='HCompV', **kwargs)
    model.train(X_train, y_train, **kwargs)

    model.edit([f'AT 2 4 0.2 {{ne.transP}}', f'AT 4 2 0.2 {{ne.transP}}'])
    model.train(X_train, y_train, **kwargs)

    for i in range(1, kwargs['gaussian_duplication_times']+1):
        model.edit([f'MU {2**i} {{*.state[2-4].mix}}'])
        model.train(X_train, y_train, **kwargs)

    results = model.test(X_test, y_test, **kwargs)

    try:
        score = results['WORD_Acc']
    except:
        score = -10
    if score < 0:
        score = -10

    if log:
        run = client.set_experiment_run(desc=EXPERIMENT_DESCRIPTION)
        run.log_hyperparameters(kwargs)
        run.log_metric('Accuracy', score)
    
    if plot_results:
        if score > 70:
            viz.visualize.generate_plots(run.id, X_test, y_test, results)
        
    return results['WORD_Acc']

### Hyperparameter space search

In [15]:
import skopt
from skopt.plots import plot_convergence
import matplotlib.pyplot as plt

from npfd import data

vf = [1, 8] 
mv = [1, 5] # Max is 10
wip = [0.0, 8]
gsf = [5, 100]
# pt = [100, 1000]
gdt = [7, 8]

space = [skopt.space.Real(vf[0], vf[1], name='variance_floor', prior='uniform'),
         skopt.space.Real(mv[0], mv[1], name='minimum_variance', prior='uniform'),
#          skopt.space.Real(wip[0], wip[1], name='word_insertion_penalty', prior='uniform'),
         skopt.space.Real(gsf[0], gsf[1], name='grammar_scale_factor', prior='uniform'),
#          skopt.space.Real(pt[0], pt[1], name='pruning_threshold', prior='log-uniform'),
#          skopt.space.Integer(gdt[0], gdt[1], name='gaussian_duplication_times')
]

@skopt.utils.use_named_args(space)
def objective(**params):
    return -1.0 * train_evaluate(gaussian_duplication_times=5,
                                 **params)

### Set up VertaAI

In [16]:
from verta import Client

HOST = 'http://127.0.0.1:3000'
PROJECT_NAME = "NPF Detector"
EXPERIMENT_NAME = "DMPS only. Manually selected files to train. Add transitions 2-4 in NE. Make dataset in each iteration 16/07/2021"
EXPERIMENT_DESCRIPTION = f"""
Search space :
- variance_floor={vf[0], vf[1]}
- minimum_variance={mv[0], mv[1]}
- word_insertion_penalty={wip[0], wip[1]}
- grammar_scale_factor={gsf[0], gsf[1]}
- gaussian_duplication_times= {gdt[0], gdt[1]}""" 

client = Client(HOST)
proj = client.set_project(PROJECT_NAME)
expt = client.set_experiment(EXPERIMENT_NAME)

connection successfully established
set existing Project: NPF Detector from personal workspace
created new Experiment: DMPS only. Manually selected files to train. Add transitions 2-4 in NE. Make dataset in each iteration 16/07/2021


Run the optimizer

In [19]:
results = skopt.gp_minimize(objective, space, n_calls=30)

best_auc = -1.0 * results.fun
best_params = results.X

INFO:root:Converting real raw files to HTK format ...
INFO:root:Generating Master Label File (Train)...
INFO:root:Generating Master Label File (Test)...
INFO:root:Initializing model...
INFO:root:/home/gfogwil/Documentos/Facultad/Tesis/models/bdb/npfd/models/HTK/misc
INFO:root:Training the model...



/home/gfogwil/Documentos/Facultad/Tesis/models/bdb/notebooks/hyperparameter-search
Pruning-On[250.0 150.0 1000.0]
 WARNING [-7324]  StepBack: File /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/data/interim/dmps_train_D_A/20161201 - bad data or over pruning
 in /home/gfogwil/Documentos/Facultad/Tesis/programs/htk/HTKTools/HERest



INFO:root:Most trained model: 3
INFO:root:Editing model 3
INFO:root:Most trained model: 4
INFO:root:Training the model...


Pruning-On[250.0 150.0 1000.0]
 WARNING [-7324]  StepBack: File /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/data/interim/dmps_train_D_A/20161201 - bad data or over pruning
 in /home/gfogwil/Documentos/Facultad/Tesis/programs/htk/HTKTools/HERest

Pruning-On[250.0 150.0 1000.0]
 WARNING [-7324]  StepBack: File /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/data/interim/dmps_train_D_A/20161201 - bad data or over pruning
 in /home/gfogwil/Documentos/Facultad/Tesis/programs/htk/HTKTools/HERest


Pruning-On[250.0 150.0 1000.0]

Pruning-On[250.0 150.0 1000.0]



INFO:root:Most trained model: 7
INFO:root:Editing model 7
INFO:root:Most trained model: 8
INFO:root:Training the model...


...

INFO:root:Most trained model: 23
INFO:root:Editing model 23
INFO:root:Most trained model: 24
INFO:root:Training the model...


Pruning-On[250.0 150.0 1000.0]


Pruning-On[250.0 150.0 1000.0]

Pruning-On[250.0 150.0 1000.0]



INFO:root:Most trained model: 27
INFO:root:Testing model: 27


Pruning-On[250.0 150.0 1000.0]

/home/gfogwil/Documentos/Facultad/Tesis/programs/htk/HTKTools/HVite -C /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/npfd/models/HTK/misc/config -H /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/models/hmm/27/macros -H /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/models/hmm/27/hmmdefs -p 0.0000000000 -s 32.5136698436 -A -T 0 -S /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/data/interim/dmps_test_D_A.scp -i /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/data/interim/results.mlf -w /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/npfd/models/HTK/misc/wdnet /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/npfd/models/HTK/misc/dict /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/npfd/models/HTK/misc/monophones 

====================== HTK Results Analysis =======================
  Date: Fri Jul 16 16:25:59 2021
  Ref : >ocumentos/Facultad/Tesis/models/bdb/data/interim/dmps_test_labels.mlf
  Rec : >gfogwil/Documentos/Facultad/Te

AttributeError: X

In [10]:
# print("""Best parameters:
# - variance_floor=%.6f
# - minimum_variance=%.6f
# - word_insertion_penalty=%.2f
# - grammar_scale_factor=%.2f
# - gaussian_duplication_times=%d""" % (results.x[0], results.x[1],
#                                       results.x[2], results.x[3],
#                                       results.x[4]))

plot_convergence(results)
plt.show()

skopt.plots.plot_evaluations(results)
plt.show()

NameError: name 'results' is not defined

In [107]:
e#.get_hyperparameters()

name: Run 1785816254977529606767
url: http://127.0.0.1:3000/personal/projects/6598fbf4-a91c-47f3-9a97-c37bd851420f/exp-runs/7bd42ab5-fdc5-4399-94f2-e4b4c297a305
date created: 2021-07-05 12:09:13.142000
date updated: 2021-07-05 12:09:55.605000
description: 
Search space :
- variance_floor=(0.001, 8)
- minimum_variance=(0.001, 5)
- word_insertion_penalty=(0.0, 8)
- grammar_scale_factor=(0, 100)
- gaussian_duplication_times= (7, 8)
tags: []
attributes: {}
id: 7bd42ab5-fdc5-4399-94f2-e4b4c297a305
experiment id: 32d83014-c4ad-46b2-bb14-48e5525bf645
project id: 6598fbf4-a91c-47f3-9a97-c37bd851420f
hyperparameters: {'gaussian_duplication_times': 8, 'variance_floor': 4.268399262928018, 'minimum_variance': 1.2281395353403621, 'word_insertion_penalty': 3.5490848564531046, 'grammar_scale_factor': 50.662569155432045}
observations: {}
metrics: {'Accuracy': 72.48}
artifact keys: []

## Best experiments

In [116]:
print(f"# de experimentos: {expt.expt_runs}")

for e in expt.expt_runs.find("metrics.Accuracy > 70"):
    params = e.get_hyperparameters()
    train_evaluate(X_train, X_test, y_train, y_test, log=False, **params)

# de experimentos: <ExperimentRuns containing 324 runs>
set existing ExperimentRun: Run 1785816255236353315084


INFO:root:Converting real raw files to HTK format ...
INFO:root:Generating Master Label File (Train)...
INFO:root:Generating Master Label File (Test)...
INFO:root:Initializing model...
INFO:root:/home/gfogwil/Documentos/Facultad/Tesis/models/bdb/npfd/models/HTK/misc
INFO:root:Training the model...



/home/gfogwil/Documentos/Facultad/Tesis/models/bdb/notebooks/hyperparameter-search
Pruning-On[250.0 150.0 1000.0]



INFO:root:Most trained model: 3
INFO:root:Editing model 3


Pruning-On[250.0 150.0 1000.0]

Pruning-On[250.0 150.0 1000.0]



...

INFO:root:Most trained model: 35
INFO:root:Editing model 35
INFO:root:Most trained model: 36
INFO:root:Training the model...


Pruning-On[250.0 150.0 1000.0]


Pruning-On[250.0 150.0 1000.0]

Pruning-On[250.0 150.0 1000.0]



INFO:root:Most trained model: 39
INFO:root:Testing model: 39


Pruning-On[250.0 150.0 1000.0]

/home/gfogwil/Documentos/Facultad/Tesis/programs/htk/HTKTools/HVite -C /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/npfd/models/HTK/misc/config -H /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/models/hmm/39/macros -H /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/models/hmm/39/hmmdefs -p 1.5116202479 -s 50.5001414153 -A -T 0 -S /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/data/interim/dmps_test_D_A.scp -i /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/data/interim/results.mlf -w /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/npfd/models/HTK/misc/wdnet /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/npfd/models/HTK/misc/dict /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/npfd/models/HTK/misc/monophones 

====================== HTK Results Analysis =======================
  Date: Mon Jul  5 20:08:13 2021
  Ref : >ocumentos/Facultad/Tesis/models/bdb/data/interim/dmps_test_labels.mlf
  Rec : >gfogwil/Documentos/Facultad/Te

UnboundLocalError: local variable 'run' referenced before assignment